In [1]:
from joblib import load
from sklearn.ensemble import VotingClassifier
from joblib import dump

from utils import TransactionDataset, FinalModelPerformance, TuneHyperParams


In [2]:
lr_model = load("./models_exports/logistic_regression_classifier.joblib")
knn_model = load("./models_exports/knn_classifier.joblib")
rf_model = load("./models_exports/random_forest_classifier.joblib")
nb_model = load("./models_exports/naive_bayes_classifier.joblib")
svc_model = load("./models_exports/support_vector_classifier.joblib")

In [3]:
data = TransactionDataset().get_training_test_split()

In [4]:
voting_classifier = VotingClassifier(
    estimators=[
        ("lr_model", lr_model),
        ("knn_model", knn_model),
        ("rf_model", rf_model),
        ("nb_model", nb_model),
        ("svc_model", svc_model)
    ],
)


In [5]:
final_model_performance = FinalModelPerformance(
    model=voting_classifier,
    data=data
)

In [6]:
final_model_performance.get_cross_validation_results()

,Metric for Training Set,Score
0,fit_time mean score,168.239557
1,score_time mean score,0.834320
2,5-fold CV F1 Score mean score,0.671330
3,5-fold CV Accuracy mean score,0.883141
4,5-fold CV Balanced Accuracy mean score,0.834376
5,5-fold CV Matthew's Correlation Coefficient me...,0.608310


In [7]:
final_model_performance.get_final_model_performance()

,Metric for Testing Set,Score
0,F1 Score,0.698451
1,Accuracy,0.896354
2,Balanced Accuracy,0.798139
3,Matthew's Correlation Coefficient,0.640204


In [9]:
dump(final_model_performance.model, "./models_exports/voting_ensemble.joblib")

['./models_exports/voting_ensemble.joblib']